# LPI Tutorial

Get mean LPI for each REW channel

In [1]:
from matplotlib import pyplot as plt
import matplotlib 
%matplotlib inline
import pandas as pd
import gdal
import os
import datetime
from datetime import timedelta
import numpy as np
import fiona
import shapely
from shapely.geometry import Point
from os.path import dirname
import glob
import rasterio
import sys
import pickle
from functools import partial
import geopandas as gp

parent_dir = dirname(dirname(os.getcwd()))
sys.path.append(os.path.join(parent_dir,'StreamflowTempModel','lib'))
rew_config = pd.read_pickle(os.path.join(parent_dir,'model_data','rew_config.p'))

# Add LPI to streams shapefile


In [2]:
streams_file = glob.glob(os.path.join(parent_dir, 'raw_data','streams_poly','*.shp'))[0]
streams = gp.read_file(streams_file)

if 'lpi' in streams.columns:
    geos = streams.geometry.values
    ids = streams.stream.values
    pts_list = []
    ids_list = []
    for i,geo in enumerate(geos):
        pts = [pair for pair in geo.coords]
        pts_list = pts_list + pts
        ids_list = ids_list + len(pts)*[ids[i]]
    src = rasterio.open(os.path.join(parent_dir,'raw_data', 'lpi', 'lpi_predicted.tif'))
    samples = np.array([item[0] for item in src.sample(pts_list)])
    samples[samples<0] = 0
    samples[samples>100]=100
    lpi_df = pd.DataFrame.from_dict({'lpi':samples, 'stream':ids_list})
    for rew_id in rew_config.keys():
        rew_config[rew_id]['lpi'] = streams.loc[streams.stream==rew_id, 'lpi'].values[0]
    pickle.dump( rew_config, open( os.path.join(parent_dir,'model_data','rew_config.p'), "wb" ) )    
    print('LPI has already been added to the streams shapefile!')
else:
    geos = streams.geometry.values
    ids = streams.stream.values
    pts_list = []
    ids_list = []
    for i,geo in enumerate(geos):
        pts = [pair for pair in geo.coords]
        pts_list = pts_list + pts
        ids_list = ids_list + len(pts)*[ids[i]]
    src = rasterio.open(os.path.join(parent_dir,'raw_data', 'lpi', 'lpi_predicted.tif'))
    samples = np.array([item[0] for item in src.sample(pts_list)])
    samples[samples<0] = 0
    samples[samples>100]=100
    lpi_df = pd.DataFrame.from_dict({'lpi':samples, 'stream':ids_list})
    new_streams = pd.DataFrame(lpi_df.groupby('stream')['lpi'].mean()).merge(streams, left_on='stream', right_on='stream')
    new_streams = gp.GeoDataFrame(new_streams)
    for rew_id in rew_config.keys():
        rew_config[rew_id]['lpi'] = new_streams.loc[new_streams.stream==rew_id, 'lpi'].values[0]
    pickle.dump( rew_config, open( os.path.join(parent_dir,'model_data','rew_config.p'), "wb" ) )
    new_streams.to_file(streams_file)

LPI has already been added to the streams shapefile!


# Filter terrain derived solar using LPI

In [3]:
streams_file = glob.glob(os.path.join(parent_dir, 'raw_data','streams_poly','*.shp'))[0]
streams = gp.read_file(streams_file)

rss = pd.DataFrame([], index=[rew_id for rew_id in rew_config.keys()], columns=np.arange(1,366,1))

# these tifs need to be generated on a weekly basis over the model domain
tifs = glob.glob(os.path.join(parent_dir, 'raw_data', 'sf_rs','*.tif'))

for raster_file in tifs:
    day = int(raster_file[-7:-4])
    src = rasterio.open(raster_file)
    samples = np.array([item[0] for item in src.sample(pts_list)])
    rsdf = pd.DataFrame.from_dict({'rs':samples, 'stream':ids_list})
    rsdf['rs_lpi'] = rsdf['rs']*(lpi_df['lpi']/100.0)
    rs = rsdf.groupby('stream')['rs_lpi'].apply(np.nanmean)
    rss.at[rs.index, day] = rs
rss = rss.T.astype(float).interpolate()*3600
rss.index = rss.index - 1
pickle.dump(rss, open(os.path.join(parent_dir, 'raw_data', 'radiation', 'rss.p'),'wb'))